`quote` block makes a `begin` block, which is an `Expr`.
An `Expr` stores expressions, evaluation of which returns the last `return` object.

In [ ]:
ex = :(println("Calculating..."); 1 + 2)

println(typeof(ex))

eval(ex)

In [ ]:
ex = quote
    println("Calculating...")
    1 + 2
end

println(typeof(ex))

eval(ex)

Just as `"$va"` interpolates `va` in the process of making a `String`, `$` interpolates variables in the process of making an `Expr`.

In [ ]:
va = 8;

In [ ]:
# This `Expr` interpolates `va`, which has been defined to be 8.
qu = quote
    1 + 2
    3 * $va
end
println(qu, " IS A ", typeof(qu), " AND EVALUATES TO ", eval(qu))

In [ ]:
try
    # This `Expr` interpolates `no`, which has not been defined.
    quote
        1 + 2
        3 * $no
    end
catch er
    println(er)
end

Every expression contained within an `Expr` must be able to execute.

In [ ]:
# This `Expr` plans to use `va`, which will be defined to be 8.
qu = quote
    1 + 2
    3 * va
end
println(qu, " IS A ", typeof(qu), " AND EVALUATES TO ", eval(qu))

In [ ]:
# This `Expr` plans to use `no`, which will not be defined.
qu = quote
    1 + 2
    3 * no
end
try
    println(qu, " IS A ", typeof(qu), " AND EVALUATES TO ", eval(qu))
catch er
    println(er)
end

`macro`s make expressions at parse time.
A `macro` takes expressions, symbols, andor literals, makes one of them, and replaces itself with it, which is then immediately evaluated.

In [ ]:
macro ma(ex)
    # This `Expr` is `ex`, which is interpolated.
    ex
end
println(
    (@macroexpand @ma 1 + 2),
    " BECOMES A ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)

In [ ]:
macro ma(ex)
    # This `Expr` contains `ex`, which will be interpolated.
    quote
        ex
    end
end
try
    println(
        (@macroexpand @ma 1 + 2),
        " BECOMES A ",
        typeof(@macroexpand @ma 1 + 2),
        " THAT EVALUATES TO ",
        (@ma 1 + 2),
        " OF TYPE ",
        typeof(@ma 1 + 2),
    )
catch er
    println(er)
end

In [ ]:
macro ma(ex)
    # This `Expr` contains `ex`, which is interpolated.
    quote
        $ex
    end
end
println(
    (@macroexpand @ma 1 + 2),
    " BECOMES A ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)

In [ ]:
macro ma(ex)
    # This is not an `Expr` but just a `String`.
    "ex"
end
println(
    (@macroexpand @ma 1 + 2),
    " BECOMES ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)

In [ ]:
macro ma(ex)
    # This is a `String` that has interpolated `ex`.
    "$ex"
end
println(
    (@macroexpand @ma 1 + 2),
    " REPLACES ITSELF WITH ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)

In [ ]:
macro ma(ex)
    # This `Expr` contains a `String` that plans to interpolate `ex`.
    quote
        "$ex"
    end
end
try
    println(
        (@macroexpand @ma 1 + 2),
        " BECOMES A ",
        typeof(@macroexpand @ma 1 + 2),
        " THAT EVALUATES TO ",
        (@ma 1 + 2),
        " OF TYPE ",
        typeof(@ma 1 + 2),
    )
catch er
    println(er)
end

In [ ]:
macro ma(ex)
    # This `Expr` contains a `String` that has interpolated `ex`.
    quote
        "$($ex)"
    end
end
println(
    (@macroexpand @ma 1 + 2),
    " BECOMES A ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)

First, variables within a macro result are classified as either local or global. A variable is considered local if it is assigned to (and not declared global), declared local, or used as a function argument name. Otherwise, it is considered global. Local variables are then renamed to be unique (using the gensym function, which generates new symbols), and global variables are resolved within the macro definition environment. 

In [ ]:
variable = "Important information"

macro ma(ex)
    quote
        variable = "LOL"
        $ex
    end
end
@ma 1 + 2
variable

In [ ]:
variable = "Important information"

macro ma(ex)
    esc(quote
        variable = "LOL"
        $ex
    end)
end
@ma 1 + 2
variable

In [ ]:
va = 1
macro zero_variable(ex)
    esc(:(va = 0; $ex))
end
println(@zero_variable 1 + 2)
va

In [ ]:
va = 1
macro zero_variable(ex)
    esc(quote
        va = 0
        $ex
    end)
end
println(@zero_variable 1 + 2)
va